In [40]:
import os, sys, glob
import numpy as np
import pydicom as dicom
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.draw import polygon
import nibabel as nib
import scipy.ndimage

"""
Preprocess script for DICOM CT scans and RTSS structure files
Specifically for Radiation Oncology CT simulation scans
"""

def read_structure(structure):
    """
    INPUT:
        structure: RTSS structure file
    OUTPUT:
        contours: a list of structures, where each structure is a dict with structure name, color, number & coordinates
    """
    contours = []
    for i in range(len(structure.ROIContourSequence)):
        contour = {}
        contour['color'] = structure.ROIContourSequence[i].ROIDisplayColor
        # contour['number'] = structure.ROIContourSequence[i].RefdROINumber
        contour['number'] = structure.StructureSetROISequence[i].ROINumber
        contour['name'] = structure.StructureSetROISequence[i].ROIName
        # if 'CTV' not in contour['name']: continue
        # assert contour['number'] == structure.StructureSetROISequence[i].ROINumber
        contour['contours'] = [s.ContourData for s in structure.ROIContourSequence[i].ContourSequence]
        contours.append(contour)
    return contours

def get_mask(contours, slices):
    """
    INPUT:
        coutours: output from read_structure; a list of structures
        slices: a list of dicom slices of CT scan corresponding to contours
    OUTPUT:
        label: a mask of the original CT scan where the values correspond to structure number
        colors: a list of colors corresponding 
    """
    z = [s.ImagePositionPatient[2] for s in slices]
    pos_r = slices[0].ImagePositionPatient[1]
    spacing_r = slices[0].PixelSpacing[1]
    pos_c = slices[0].ImagePositionPatient[0]
    spacing_c = slices[0].PixelSpacing[0]
    
    print(z)

    label = np.zeros_like(image, dtype=np.uint8)
    for con in contours:
        num = int(con['number'])
        for c in con['contours']:
            nodes = np.array(c).reshape((-1, 3))
            assert np.amax(np.abs(np.diff(nodes[:, 2]))) == 0
            zNew = [round(elem,0) for elem in z ] 
            try:
                z_index = z.index(round(nodes[0, 2], 1))
            except ValueError:
                try:
                    z_index = zNew.index(round(nodes[0, 2], 0))
                except ValueError:
                    z_index = (np.abs(z - nodes[0, 2])).argmin()
            # z_index = z.index(np.around(nodes[0, 2],1))
            r = (nodes[:, 1] - pos_r) / spacing_r
            c = (nodes[:, 0] - pos_c) / spacing_c
            rr, cc = polygon(r, c)
            label[rr, cc, z_index] = num
    colors = tuple(np.array([con['color'] for con in contours]) / 255.0)
    return label, colors

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices], axis=-1)
    # Convert to int16 (from sometimes int16),
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0

    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):

        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        if slope != 1:
            image[:, :, slice_number] = slope * image[slice_number].astype(np.float64)
            image[:, :, slice_number] = image[slice_number].astype(np.int16)

        image[:, :, slice_number] += np.int16(intercept)

    return np.array(image, dtype=np.int16)

def resample(image, slices, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array(list(slices[0].PixelSpacing) + [slices[0].SliceThickness], dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [47]:
dataset_directory = './data/Head-Neck-PET-CT'
output_directory = os.path.join(dataset_directory+"_preprocessed")
patient_list = [os.path.join(dataset_directory, name) for name in os.listdir(dataset_directory) if os.path.isdir(os.path.join(dataset_directory, name))]

for patient in patient_list:
    print(patient)
    
    if os.path.exists(os.path.join(output_directory, os.path.basename(patient), "CT.npy")): continue
    
    for subdir, dirs, files in os.walk(patient):
        dcms = glob.glob(os.path.join(subdir, "*.dcm"))
        if len(dcms) == 1:
            # structure = dicom.read_file(os.path.join(subdir, files[0]))
            # contours = read_structure(structure)
            print('RTStructure detected')
        elif len(dcms) > 1:
            slices = [dicom.read_file(dcm) for dcm in dcms]
            slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
            if slices[0].Modality == 'CT':
                image = get_pixels_hu(slices) #convert to hounsfeld unit
                image, new_spacing = resample(image, slices) #resample to 1x1x1
            elif slices[0].Modality == 'PT':
               pet_image = np.stack([s.pixel_array for s in slices], axis=-1)
               pet_image, new_spacing = resample(pet_image, slices)

    # label, colors = get_mask(contours, slices) #colors can be used to visualize using pyplot

#     plt.figure(figsize=(15, 15))
#     for i in range(9):
#         plt.subplot(3, 3, i + 1)
#         plt.imshow(image[..., i + 50], cmap="gray")
#         plt.axis('off')
#     plt.imshow()
    if not os.path.isdir(os.path.join(output_directory, os.path.basename(patient))):
        os.makedirs(os.path.join(output_directory, os.path.basename(patient)))
    
    np.save(os.path.join(output_directory, os.path.basename(patient), "CT.npy"), image)
    np.save(os.path.join(output_directory, os.path.basename(patient), "PET.npy"), pet_image)
#     np.save(os.path.join(output_directory, os.path.basename(patient)+"_label.npy"), label)
    break

./data/Head-Neck-PET-CT/HN-CHUM-030
RTStructure detected
RTStructure detected


In [49]:
import nibabel as nib

ct_vol = np.load('./data/Head-Neck-PET-CT_preprocessed/HN-CHUM-030/CT.npy')
pet_vol = np.load('./data/Head-Neck-PET-CT_preprocessed/HN-CHUM-030/PET.npy')

# real_resize_factor = np.asarray(ct_vol.shape)/np.asarray(pet_vol.shape)
# pet_vol_new = scipy.ndimage.interpolation.zoom(pet_vol, real_resize_factor, mode='nearest')

ct_nii = nib.Nifti1Image(ct_vol, np.eye(4))
pet_nii = nib.Nifti1Image(pet_vol_new, np.eye(4))
ct_nii.to_filename('ct.nii')
pet_nii.to_filename('pet.nii')

In [2]:
data_path = "./data/Head-Neck-PET-CT/HN-CHUM-001/08-27-1885-PANC. avec C.A. SPHRE ORL   tte et cou  -TP-74220"
ct_path = os.path.join(data_path, '3-StandardFull-07232')
pet_path = os.path.join(data_path, '4-TETECOUAC2D-27812')

ct_dcms = glob.glob(os.path.join(ct_path, "*.dcm"))
pet_dcms = glob.glob(os.path.join(pet_path, "*.dcm"))

ct_slices = [dicom.read_file(dcm) for dcm in ct_dcms]
ct_slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
pet_slices = [dicom.read_file(dcm) for dcm in pet_dcms]
pet_slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))

ct_vol = np.stack([s.pixel_array for s in ct_slices], axis=-1)
pet_vol = np.stack([s.pixel_array for s in pet_slices], axis=-1)

ct_vol, new_spacing = resample(ct_vol, ct_slices)
pet_vol, new_spacing = resample(pet_vol, pet_slices)

print('ct_vol: ', ct_vol.shape)
print('pet_vol: ', pet_vol.shape)


ct_vol:  (500, 500, 338)
pet_vol:  (450, 450, 298)


numpy.int16